## Interactive website to generate publication-ready figures
- Goal: Create a website that scientists can use to upload their sequence data and generate publication ready visualizations (for non-coding familiar users)

### Requirements:
1. Set up environment
- Language: Python
- Libraries: flask, pandas, matplotlib, seaborn, plotly, scikit-learn
2. Frontend: A user-friendly interface for data upload, parameter selection, and figure customization.
- Framework: React (with libraries like shadcn/ui for UI components)
- File upload handling
- Interactive figure customization
3. Backend: Data processing and visualization generation.
- Language: Python (Flask or FastAPI)
- Libraries: Seaborn, Matplotlib, Plotly
- Handling CSV/TXT data uploads
4. Figure Generation:
- Heatmaps: Seaborn/Matplotlib
- Volcano & MA Plots: Matplotlib
- PCA: Scikit-learn for computation, Plotly for interactive visuals
5. Deployment:
- Docker container for reproducibility
- Local server

### Step 1: Set up the environment
- Run in terminal under C:\repos
- use of the following is useful for small Python only projects:
    - python -m venv venv
    - venv\Scripts\activate
- need to use conda activate venv if wanting to implement other languages and for ease of use
```
mkdir seq-visualizer
cd seq-visualizer
python -m venv venv
venv\Scripts\activate #on linux: source venv/bin/activate
pip install flask pandas matplotlib seaborn plotly scikit-learn
```

### Step 2: Create the frontend (HTML)
- Create a file "index.html" within a new "/static" folder
```
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Transcriptomics Visualizer</title>
    <script src="https://cdn.jsdelivr.net/npm/plotly.js-dist@2.19.1"></script>  <!-- Add Plotly.js -->
</head>
<body>
    <h1>Transcriptomics Visualizer</h1>

    <!-- Generate Heatmap Section -->
    <h2>Generate Heatmap</h2>
    <input type="file" id="heatmapFileInput" accept=".csv" required>
    <button onclick="generateHeatmap()">Generate Heatmap</button>
    <div id="heatmap"></div>  <!-- Placeholder for heatmap -->

    <!-- Generate Volcano Plot Section -->
    <h2>Generate Volcano Plot</h2>
    <input type="file" id="volcanoFileInput" accept=".csv" required>
    <button onclick="generateVolcanoPlot()">Generate Volcano Plot</button>
    <div id="volcano"></div>  <!-- Placeholder for volcano plot -->

    <script>
        // Function to generate the heatmap
        async function generateHeatmap() {
            let fileInput = document.getElementById("heatmapFileInput").files[0];
            if (!fileInput) {
                alert("Please select a file for the heatmap.");
                return;
            }

            let formData = new FormData();
            formData.append("file", fileInput);

            // Upload and generate heatmap in one step
            let response = await fetch("/heatmap", { method: "POST", body: formData });
            let plotData = await response.json();

            console.log("Heatmap Data:", plotData);  // Log the response

            if (plotData.error) {
                alert(plotData.error);  // Show error message if any
                return;
            }

            // Render the heatmap using Plotly
            Plotly.react('heatmap', plotData.data, plotData.layout);
        }

        // Function to generate the volcano plot
        async function generateVolcanoPlot() {
            let fileInput = document.getElementById("volcanoFileInput").files[0];
            if (!fileInput) {
                alert("Please select a file for the volcano plot.");
                return;
            }

            let formData = new FormData();
            formData.append("file", fileInput);

            try {
                // Upload and generate volcano plot in one step
                let response = await fetch("/volcano", { method: "POST", body: formData });
                let result = await response.json();

                console.log("Volcano Plot Data:", result);  // Log the response

                if (result.error) {
                    alert(result.error);  // Show error message if any
                    return;
                }

                // Render the volcano plot using Plotly
                Plotly.react('volcano', result.data, result.layout);
            } catch (error) {
                console.error("Error generating volcano plot:", error);
                alert("An error occurred while generating the volcano plot.");
            }
        }
    </script>
</body>
</html>
```

### Step 3A: Create the backend (Flask Server)
- Create a file "app.py" and add the following
```
import os
import io
import pandas as pd
import plotly.graph_objects as go
from flask import Flask, request, jsonify, send_from_directory

app = Flask(__name__)

# Serve index.html
@app.route('/')
def serve_index():
    return send_from_directory("static", "index.html")

# Function to handle CSV parsing with error handling for encoding
def parse_csv(file):
    try:
        df = pd.read_csv(file, index_col=0, encoding='utf-8')
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(file, index_col=0, encoding='latin1')
        except Exception as e:
            raise Exception(f'Failed to read CSV file: {str(e)}')
    return df

# Heatmap route
@app.route('/heatmap', methods=['POST'])
def generate_heatmap():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    try:
        # Read file directly into memory
        df = parse_csv(file)
    except Exception as e:
        return jsonify({'error': str(e)}), 400

    if df.empty:
        return jsonify({'error': 'Uploaded CSV is empty or invalid.'}), 400

    # Create heatmap
    fig = go.Figure(data=go.Heatmap(
        z=df.values.tolist(),
        colorscale='Viridis',
        colorbar=dict(title='Expression')
    ))
    fig.update_layout(
        title='Gene Expression Heatmap',
        xaxis_title='Genes',
        yaxis_title='Samples'
    )

    return jsonify(fig.to_dict())

# Volcano plot route
@app.route('/volcano', methods=['POST'])
def generate_volcano():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    try:
        # Read file directly into memory
        df = parse_csv(file)
    except Exception as e:
        return jsonify({'error': str(e)}), 400

    # Ensure necessary columns exist
    if 'log2FoldChange' not in df.columns or '-log10(pvalue)' not in df.columns:
        return jsonify({'error': 'CSV must contain "log2FoldChange" and "-log10(pvalue)" columns.'}), 400

    # Create volcano plot
    fig = go.Figure()

    # Significance threshold (e.g., p < 0.05)
    significant = df['pvalue'] < 0.05

    fig.add_trace(go.Scatter(
        x=df.loc[significant, 'log2FoldChange'].tolist(),
        y=df.loc[significant, '-log10(pvalue)'].tolist(),
        mode='markers',
        marker=dict(color='red', size=8),
        name='Significant (p < 0.05)'
    ))

    fig.add_trace(go.Scatter(
        x=df.loc[~significant, 'log2FoldChange'].tolist(),
        y=df.loc[~significant, '-log10(pvalue)'].tolist(),
        mode='markers',
        marker=dict(color='blue', size=8),
        name='Not Significant (p >= 0.05)'
    ))

    fig.update_layout(
        title="Volcano Plot",
        xaxis_title="Log2 Fold Change",
        yaxis_title="-Log10 P-value",
        showlegend=True
    )

    return jsonify(fig.to_dict())

if __name__ == '__main__':
    app.run(debug=True)
```

### Step 3B: Test the backend
- Create "heatmap_data.csv" in terminal or Visual Studio Code
```
SampleID,GeneA,GeneB,GeneC,GeneD,GeneE
Sample1,5.1,2.3,3.5,4.8,3.2
Sample2,7.4,6.1,5.9,7.3,6.8
Sample3,3.2,2.1,4.7,3.9,4.5
Sample4,6.5,7.2,5.8,6.9,6.4
Sample5,4.1,3.5,4.8,5.2,4.7
```
- Run the flask server in terminal
```
python app.py
```
- Save code below as new "generate_volcano_data.py" file
```
import pandas as pd
import numpy as np

# Set seed for reproducibility
np.random.seed(42)

# Generate 100 random genes
genes = [f"Gene{i}" for i in range(1, 101)]

# Simulate log2 fold changes
# Cluster some genes around zero (non-significant), and some with large positive/negative values
log2fc_non_sig = np.random.normal(0, 0.5, size=70)  # Non-significant genes clustered around 0
log2fc_sig = np.random.uniform(-3, 3, size=30)      # Significant genes with larger FC
log2fc = np.concatenate([log2fc_non_sig, log2fc_sig])

# Simulate p-values: Most are non-significant (p > 0.05), a smaller fraction are significant
pvalue_non_sig = np.random.uniform(0.05, 0.5, size=70)  # Non-significant genes
pvalue_sig = np.random.uniform(0.0001, 0.05, size=30)   # Significant genes
pvalues = np.concatenate([pvalue_non_sig, pvalue_sig])

# Calculate the -log10(pvalue) to plot as the y-values for the volcano plot
negLog10_pvalues = -np.log10(pvalues)

# Create dataframe
df = pd.DataFrame({
    "gene": genes,
    "log2FoldChange": log2fc,
    "pvalue": pvalues,
    "-log10(pvalue)": negLog10_pvalues
})

# Save to CSV
df.to_csv("volcano_data.csv", index=False)

print(df.head())  # Display the first few rows of the generated data
```
- Open a new terminal and run the code to generate the volcano plot data
```
python generate_volcano_data.py
```
- Visualize the generated data file in terminal
```
type volcano_data.csv
```

### Step 4: Generate figures
- Make sure there is a terminal open running app.py
- Open a web browser and navigate to: http://127.0.0.1:5000/
- You should be able to see the following options:
    1. Generate Heatmap
    2. Generate Volcano Plot
- Click "Choose File" under either section above and use corresponding "Generate ..." button to visualize plot
- The resulting interactive plot should appear, buttons at the top right of the plot allow for manipulation and options to save the plot
    

### Step 5: Deployment
- Use Docker to create an image of the environment and make it reproducible for others to use
- Save code below as "Dockerfile"
```
### Dockerfile

# Use an official Python runtime as a parent image
FROM python:3.11-slim

# Set the working directory in the container
WORKDIR /app

# Copy the whole project to the container
COPY . /app/

# Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Expose port 5000
EXPOSE 5000

# Set the environment variable for Flask
ENV FLASK_APP=app.py

# Set Flask to development mode
ENV FLASK_ENV=development

# Command to run the Flask app
CMD ["flask", "run", "--host=0.0.0.0", "--port=5000"]
```
- Save code below as "requirements.txt"
```
Flask==3.0.0
pandas==2.2.0
matplotlib==3.8.2
seaborn==0.13.2
plotly==5.18.0
scikit-learn==1.4.1.post1
```
- Save code below as "docker-compose.yml"
```
services:
  web:
    build: .
    ports:
      - "5000:5000"
    environment:
      - FLASK_ENV=development
    command: flask run --host=0.0.0.0 --port=5000
```
- Save code below as ".dockerignore"
```
# Ignore Git files and directories
.git
.gitignore

# Ignore node_modules (if you’re using Node.js)
node_modules/

# Ignore any temporary files and directories
*.log
*.tmp
*.swp

# Ignore environment variables and configuration files
.env
*.env

# Ignore Dockerfile itself and any scripts related to local dev
Dockerfile
docker-compose.yml
scripts/

# Ignore build artifacts
build/
dist/

# Ignore IDE/editor specific files (VS Code, JetBrains, etc.)
.vscode/
.idea/
```
- Run the following code to create docker image in terminal:
```
docker-compose up
# if image isn't already created, it will be built before running in container
# if need to modify the script and rebuild the image, run the following line
# docker-compose up --build
```
- After building, the image should open in a new container and the webpage can be accessed in the web browser by navigating to: http://127.0.0.1:5000/
    - Can also open the image status in Docker Desktop and find the clickable link to the webpage

### How to allow others to use the same environment:
- User must have Docker installed: https://www.docker.com/get-started/
- Include the following files in the appropriate directory structure:
```
C:\repos\seq-visualizer
│
├── static
│   ├── index.html
├── .dockerignore
├── app.py
├── docker-compose.yml
├── Dockerfile
└── requirements.txt
```
- After setting up directory, run the following code in terminal to build docker image
```
# python -m venv venv # only need to create this once
# on windows: venv\Scripts\activate 
OR
# on linux: source venv/bin/activate

wsl # open linux box on windows
docker-compose up
# if image isn't already created, it will be built before running in container
# if need to modify the script and rebuild the image, run the following line
# docker-compose up --build
```
- Open web browser and navigate to: http://127.0.0.1:5000/